In [1]:
import os
import re
import numpy as np
from openai import OpenAI
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any, Dict, List
from google import genai
from google.genai.errors import APIError
from dotenv import load_dotenv
import time

In [ ]:
load_dotenv() 

# Verification
print(os.getenv("GEMINI_API_KEY"))

In [3]:
df_raw = pd.read_csv(
    "wide_filled.csv"
)

df = df_raw.copy()
df["YEARWEEK"] = df["YEARWEEK"].astype(int)
df = df.sort_values("YEARWEEK")
s = df['YEARWEEK'].astype(str)

# Monday of each ISO year-week
df['week_start'] = pd.to_datetime(s + '1', format='%G%V%u')
train = df.iloc[0:468, 11].to_numpy()
test = df.iloc[468:520, 11].to_numpy()

In [4]:
def _serialize_series(arr, precision=2):
    """
    Turn a numeric iterable into a comma-separated string.
    - precision: exact number of digits after the decimal point
    - NaN / missing -> empty token (i.e., consecutive commas)
    - Example: _serialize_series([0.1, np.nan, 2], precision=4) -> '0.1000,,2.0000'
    """
    a = np.asarray(arr, dtype=float)
    toks = []
    fmt = "{:." + str(precision) + "f}"
    for x in a:
        if np.isnan(x):
            toks.append("")
            continue
        s = fmt.format(x)
        # normalize negative zero like '-0.0000' -> '0.0000'
        if float(s) == 0.0 and s.startswith('-'):
            s = s[1:]
        toks.append(s)
    return ",".join(toks)

def _parse_numbers(text: str, expect_n: int) -> List[float]:
    """MOCK: Parses a comma-separated string into a list of floats,
    handling empty tokens (e.g., ",,").
    """
    try:
        parts = text.strip().split(',')
        if len(parts) != expect_n:
            print(f"Warning: Expected {expect_n} values, got {len(parts)} parts in: {text[:50]}...")
            # Attempt to fill with NaNs if necessary, or just process what we have
            # For this context, we will try to parse up to expect_n elements.
        
        results = []
        for part in parts[:expect_n]:
            try:
                # Replace empty string with NaN
                results.append(float(part.strip()) if part.strip() else np.nan)
            except ValueError:
                # Handle non-numeric text if any slips through (shouldn't with good prompt)
                results.append(np.nan)

        # Pad with NaNs if fewer than expected elements were parsed
        while len(results) < expect_n:
             results.append(np.nan)
             
        return results
        
    except Exception as e:
        print(f"Error parsing text '{text[:50]}...': {e}")
        return [np.nan] * expect_n


In [5]:
load_dotenv()
def gemini_forecast(
    train: List[float] | np.ndarray,
    test: List[float] | np.ndarray,
    model: str = "gemini-2.5-flash",
    *,
    n_samples: int = 1,
    precision: int = 2,
    max_tokens_per_step: int = 8,
    prompt_level: int = 3,
    api_key: str = None,
) -> Dict[str, Any]:
    """
    Forecast len(test) steps using the Google GenAI SDK (Gemini) with a SINGLE prompt string.

    The functionality, input/output, and prompt structure are preserved.
    """
    # 1. Initialize Gemini Client
    try:
        # The SDK automatically uses the GEMINI_API_KEY environment variable.
        client = genai.Client(api_key=api_key or os.getenv("GEMINI_API_KEY"))
    except Exception as e:
        print(f"Error initializing Gemini Client: {e}")
        raise

    train = np.asarray(train, dtype=float)
    test  = np.asarray(test,  dtype=float)
    steps = int(test.size)
    
    # Early exit for empty test set (result structure preserved)
    if steps <= 0:
        return {
            "preds_mean": np.array([]),
            "rmse": np.nan,
            "all_preds": np.empty((0, 0)),
            "train": train,
            "test": test,
            "raw_texts": [],
            "config": {
                "model": model,
                "n_samples": int(n_samples),
                "precision": int(precision),
                "max_tokens_per_step": int(max_tokens_per_step),
                "prompt_level": int(prompt_level),
            },
            "meta": {"steps": steps, "prompt": ""},
        }

    seq = _serialize_series(train, precision=precision)

    # Clamp prompt_level to [1, 3]
    level = max(1, min(3, int(prompt_level)))

    # --- Build prompt in layers (identical to original) ---
    parts = []

    # Level 1: Task description
    parts.append(
        "TASK DESCRIPTION:\n"
        "You are forecasting future values for a univariate time series.\n"
        "The goal is to predict future values given the historical data.\n"
        "***IMPORTANT: If the historical input sequence contains two consecutive commas (e.g., '1.23,,1.25'), the value between them is missing.***\n"
    )

    # Level 2: Data context (ILI)
    if level >= 2:
        parts.append(
            "\nDATA CONTEXT:\n"
            "- The sequence represents *weekly Percentage of Visits for Influenza-Like Illness (ILI).* \n"
            "- Each value is the percent of all healthcare visits attributed to ILI in a given week.\n"
            "- The historical sequence you see includes data up to around September 2024.\n"
        )

    # Level 3: Modeling guidance
    if level >= 3:
        parts.append(
            "\nMODELING GUIDANCE:\n"
            "- When forecasting, explicitly consider:\n"
            "  • Long-term trend (overall increase, decrease, or stability over multiple seasons).\n"
            "  • Seasonality (typical winter influenza peaks and off-season lows each year).\n"
            "  • Randomness and noise (week-to-week fluctuations and irregular spikes).\n"
            "- Use these components implicitly in your reasoning when extending the series.\n"
        )

    # Output rules (always included)
    max_output_tokens = steps * max_tokens_per_step
    parts.append(
        "\nOUTPUT RULES:\n"
        " - Output ONLY the next values as a comma-separated list.\n"
        " - No spaces, no text, no explanations.\n\n"
        f"INPUT SEQUENCE (comma-separated):\n{seq}\n\n"
        f"TASK: Predict the next {steps} weekly ILI percentage values.\n"
        f"OUTPUT FORMAT EXAMPLE (for 3 steps): 1.23,1.25,1.27\n"
        "Remember: Output ONLY the comma-separated numbers."
    )

    prompt = "".join(parts)

    samples = max(1, int(n_samples))
    raw_texts, preds_list = [], []

    # 2. Call Gemini API for each sample
    for _ in range(samples):
        raw = ""
        try:
            # Use generate_content for text generation.
            response = client.models.generate_content(
                model=model,
                contents=prompt
            )
            raw = response.text.strip()
            
        except APIError as e:
            raw = f"API_ERROR: {e}"
            print(f"Gemini API Error: {e}")
        except Exception as e:
            raw = f"GENERIC_ERROR: {e}"
            print(f"Generic Error: {e}")
            
        raw_texts.append(raw)
        
        # 3. Parse and store predictions
        preds_list.append(np.asarray(_parse_numbers(raw, steps), dtype=float))

    # 4. Calculate final metrics
    all_preds = np.stack(preds_list, axis=0)                  # (n_samples, steps)
    preds_mean = np.nanmean(all_preds, axis=0) # Use nanmean to handle NaNs from parsing errors
    
    # Calculate RMSE, handling potential NaNs in the prediction mean
    if np.any(np.isnan(preds_mean)):
        valid_indices = ~np.isnan(preds_mean)
        if np.any(valid_indices):
            rmse = float(np.sqrt(np.mean((preds_mean[valid_indices] - test[valid_indices]) ** 2)))
        else:
            rmse = np.nan
    else:
        rmse = float(np.sqrt(np.mean((preds_mean - test) ** 2))) # RMSE vs true test

    # 5. Return the result dictionary
    return {
        "preds_mean": preds_mean,
        "rmse": rmse,
        "all_preds": all_preds,
        "train": train,
        "test": test,
        "raw_texts": raw_texts,
        "config": {
            "model": model,
            "n_samples": samples,
            "precision": int(precision),
            "max_tokens_per_step": int(max_tokens_per_step),
            "prompt_level": level,
        },
        "meta": {
            "steps": steps,
            "prompt": prompt,
        },
    }

In [6]:
def gemini_forecast_plot(
    res,
    time_points,
    title="Gemini Forecast (train, test truth, and prediction)",
    xlabel="Time",
    ylabel="Value",
    ax=None,
    legend_loc="best",
):
    """
    Plot GPT forecast results (no train predictions or CIs).
    Expects `res` from gpt_forecast_llmtime(...) that contains:
        - res["train"]      : 1D array of train values
        - res["test"]       : 1D array of test values (truth)
        - res["preds_mean"] : 1D array of predictions for test horizon

    Args:
        res         : dict from gpt_forecast_llmtime (or similarly shaped)
        time_points : 1D array-like of length len(train) + len(test)
        title/xlabel/ylabel : plot labels
        ax          : optional matplotlib Axes
        legend_loc  : legend location

    Returns:
        (fig, ax)
    """
    import numpy as np
    import matplotlib.pyplot as plt

    train = np.asarray(res["train"], dtype=float)
    test  = np.asarray(res["test"], dtype=float)
    y_pred_test = np.asarray(res["preds_mean"], dtype=float)

    n_train = train.shape[0]
    n_test  = test.shape[0]
    if y_pred_test.shape[0] != n_test:
        raise ValueError(f"preds_mean length ({y_pred_test.shape[0]}) must equal test length ({n_test}).")

    # Coerce time_points to 1D numpy array, accept pandas Index/Series if present
    try:
        import pandas as pd  # optional
        if isinstance(time_points, (pd.Series, pd.Index)):
            x = np.asarray(time_points.to_numpy()).ravel()
        else:
            x = np.asarray(time_points).ravel()
    except Exception:
        x = np.asarray(time_points).ravel()

    if x.shape[0] != n_train + n_test:
        raise ValueError(f"`time_points` must have length {n_train + n_test} (got {x.shape[0]}).")

    x_train = x[:n_train]
    x_test  = x[n_train:]

    created_fig = False
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 5))
        created_fig = True
    else:
        fig = ax.figure

    # Plot true series
    ax.plot(x_train, train, label="Train (true)")
    ax.plot(x_test,  test,  label="Test (true)")

    # Plot GPT predictions on test horizon
    ax.plot(x_test, y_pred_test, "--", label="Test (Gemini prediction)")

    # Vertical separator at train/test boundary (draw at last train time if exists)
    if n_train > 0:
        ax.axvline(x_train[-1], linestyle=":", alpha=0.6)

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend(loc=legend_loc)
    fig.tight_layout()

    return fig, ax

In [ ]:
train = df.iloc[260:468, 11].to_numpy()
test = df.iloc[468:520, 11].to_numpy()

all_res = []
for i in range(50):
    time.sleep(10)
    res = gemini_forecast(train=train, test=test)
    all_res.append(res)

import pickle
with open("all_res.pkl", "wb") as f:
    pickle.dump(all_res, f)

In [7]:
res = gemini_forecast(train, test, model = "gemini-2.5-flash", prompt_level = 3)

In [ ]:
base_seed = 12345
df_half = df.copy()
df_half = df_half.iloc[len(df_half) // 2 :].reset_index(drop=True)

n_train = 208
missing_rates = np.round(np.arange(0.0, 0.61, 0.2), 2)   # 0.0 ... 0.8

prompt_level = 3

region_cols = [c for c in df_half.columns if c not in ["YEARWEEK", "week_start"]]
# region_cols = region_cols[:2].copy()
# Make sure it's truly a DataFrame (not Series), with explicit index/columns names
results_5_missing = pd.DataFrame(
    index=pd.Index(missing_rates, name="missing_rate"),
    columns=pd.Index(region_cols, name="region"),
    dtype=object
)

for region_idx, region in enumerate(region_cols):
    y = df_half[region].to_numpy(dtype=float)
    train_full = y[:n_train].copy()
    test_full  = y[n_train:].copy()
    assert np.isfinite(train_full).all(), f"Train has NaNs for region {region}."

    for r_idx, rate in enumerate(missing_rates):
        rng = np.random.default_rng(base_seed + region_idx * 10_000 + r_idx)

        train_masked = train_full.copy()
        k = int(round(rate * len(train_masked)))
        if k > 0:
            drop_idx = rng.choice(len(train_masked), size=k, replace=False)
            train_masked[drop_idx] = np.nan
            # ensure at least 2 observed remain
            obs_count = int(np.isfinite(train_masked).sum())
            if obs_count < 2:
                to_restore = rng.choice(drop_idx, size=(2 - obs_count), replace=False)
                train_masked[to_restore] = train_full[to_restore]

        res = gemini_forecast(
            train=train_masked,
            test=test_full,
            prompt_level = prompt_level
        )

        time.sleep(10)

        # POSitional assignment avoids the "Incompatible indexer with Series" issue
        results_5_missing.iat[r_idx, region_idx] = res

    print(region)

output_filename = f"results_p{prompt_level}_missing.pkl.gz"
results_5_missing.to_pickle(output_filename)

In [15]:
res = gemini_forecast(
            train=train_masked,
            test=test_full,
            prompt_level = 3
        )

In [ ]:
results_5_missing.iat[0, 2]

In [ ]:
fig, ax = gemini_forecast_plot(
    results_5_missing.iat[3, 8],
    time_points=df_half["week_start"],      # length = n_train + n_test
    xlabel="Week",
    ylabel="Value",
	title=None
)

In [12]:
def plot_rmse_boxplots(results_df: pd.DataFrame, title: str = 'Distribution of Model RMSE by Missing Rate'):
    """
    Generates side-by-side box plots for the RMSE distribution across different 
    missing rate rows in the input DataFrame.

    The function iterates through each row (missing rate), extracts the 'rmse'
    from the structured object in each cell, ignores NaN values, and plots the
    resulting distribution.

    Args:
        results_df: The DataFrame (e.g., results_5_missing) where rows are missing rates,
                    columns are regions, and cells contain result objects (e.g., {'rmse': 0.5}).
        title: The title for the generated plot.
    """
    # 1. Apply the extraction function to all cells
    rmse_data = results_df.applymap(extract_rmse)

    # 2. Prepare data for plotting
    data_to_plot = []
    labels = []

    # Iterate over the rows (missing rates)
    for rate, series in rmse_data.iterrows():
        # Drop NaN values (where forecasts failed) and convert to a list
        clean_data = series.dropna().tolist()
        
        # Ensure there is data to plot for this rate
        if clean_data:
            data_to_plot.append(clean_data)
            # Format the label nicely (e.g., 0.2 -> 20%)
            labels.append(f"{rate * 100:.0f}% Missing")

    if not data_to_plot:
        print("Error: No valid RMSE data was extracted for plotting.")
        return

    # 3. Generate the box plots
    plt.figure(figsize=(12, 6))

    # Define colors for better visualization
    colors = plt.cm.viridis(np.linspace(0, 1, len(data_to_plot)))
    
    bp = plt.boxplot(data_to_plot, 
                     patch_artist=True, 
                     labels=labels,
                     medianprops={'color': 'black'},
                     flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red', 'alpha': 0.6})

    # Color the boxes
    for box, color in zip(bp['boxes'], colors):
        box.set_facecolor(color)
        box.set_alpha(0.7)

    # Add titles and labels
    plt.title(title, fontsize=14)
    plt.xlabel(results_df.index.name.replace('_', ' ').title() if results_df.index.name else 'Grouping Variable', fontsize=12)
    plt.ylabel('Root Mean Square Error (RMSE) Distribution', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.show()
    
def extract_rmse(result_object: Any) -> float:
    """
    Safely extracts the RMSE value from a single forecast result object.

    Args:
        result_object: The content of a single cell in the results_5_missing DataFrame.
                       Expected to be a dictionary, or None/NaN if the forecast failed.

    Returns:
        The extracted RMSE value as a float, or np.nan if extraction fails.
    """
    if result_object is None or pd.isna(result_object):
        return np.nan
    try:
        # Assumes the structure is a dictionary with a key named 'rmse'
        rmse_value = result_object.get('rmse')
        # Ensure the value itself is a valid number before returning
        if isinstance(rmse_value, (int, float)) and not np.isnan(rmse_value):
            return float(rmse_value)
        return np.nan
    except AttributeError:
        # Handles cases where result_object is not a dictionary
        return np.nan
    except Exception:
        # Catch any other unexpected errors during extraction
        return np.nan

In [ ]:
plot_rmse_boxplots(results_5_missing)